In [9]:
import newspaper
from newspaper import Article
import json

In [14]:
cnn_paper = newspaper.build('https://www.cnn.com')

In [19]:
i = 0
frontier = []
for article in cnn_paper.articles:
    frontier.append(article.url)
    i += 1
    if i == 50:
        break

50